In [1]:
%%capture 
!pip install h2o==3.40.0.1

In [2]:
import numpy as np
import pandas as pd
import h2o

In [3]:
def generate_artificial_peptide(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, max_length=22) -> str:
    """
    Generate an artificial peptide based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
        
    Returns:
    -------
    str
        Generated artificial peptide
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    out_str = ''
    for i in range(len(list_of_probabilities)):
        # make synthetic signal peptide
        artificial_amino_acid = list(np.random.choice(amino_acids, 1, p=list_of_probabilities[i]))

        if artificial_amino_acid == ['-']: 
            break

        out_str += artificial_amino_acid[0]
    return out_str

def add_dunder_tail(peptide:str , max_lenght:int=22 ): 
    '''Adds a tail if a peptide is shorter than the specified max_len.
    '''
    if len(peptide) < max_lenght: 
        difference = max_lenght - len(peptide)
        sequence = peptide + ('-'*difference)
    else: 
        sequence = peptide
        
    return sequence      

def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=50) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df


def split_peptides_sequences(df_signalPP: pd.DataFrame) -> pd.DataFrame:
    '''
    Splits the amino acid sequences into individual amino acids for each position.

    Parameters:
    -----------
    df_signalPP: pandas.DataFrame
        A DataFrame containing the amino acid sequences.

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the split amino acid sequences.
    '''
    # Initialize an empty list to store the split sequences.
    peptides_split = []

    # Split each amino acid sequence into individual amino acids.
    for k, v in df_signalPP.iterrows():
        sequence = []
        for seq in v['sequence']:
            sequence.append(seq)
        peptides_split.append(sequence)

    # Convert the list of split sequences into a DataFrame and fill NaN values with '-'.
    new_peptides = pd.DataFrame(peptides_split)
    new_peptides = new_peptides.fillna('-')

    return new_peptides


def signal_peptide_predictor(list_of_probabilities: list, amino_acids: str, 
                            n_peptides: int, 
                            number_of_iterations: int, 
                            best_model: h2o.estimators, 
                            training_column_name:str = 'MM_N_peptide_abundance', 
                            max_len_of_signal_peptides:int = 30,
                            length_of_return_df = 500, 
                            one_hot_encode:bool = True) -> pd.DataFrame:
    '''
    Predicts the best signal peptides from a given number of iterations.

    Parameters:
    -----------
    list_of_probabilities: list
        A list of probabilities.
    amino_acids: str
        A string containing the amino acids used to generate the peptides.
    n_peptides: int
        The number of peptides to generate.
    number_of_iterations: int
        The number of iterations to run the predictor.
    best_model: h2o.estimators
        The trained model to use for prediction.
    training_column_name : str
        The name of the column that the model has been trained on.
    max_len_of_signal_peptides : int
        The maximum lenght of the signal peptides you want to be generated 

    Returns:
    --------
    pandas.DataFrame
        A DataFrame containing the predicted signal peptides.
    '''
    # Initialize an empty DataFrame to store the predicted peptides.
    data = pd.DataFrame()
    
    # Generate and predict peptides for each iteration.
    for i in range(0, number_of_iterations):
        # Generate new peptides.
        new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, amino_acids, n_peptides=n_peptides, max_len=max_len_of_signal_peptides)
        
        # Split the peptides into sequences.
        new_TO_NATURE_peptides = split_peptides_sequences(new_TO_NATURE_peptides)

        if one_hot_encode:
            new_TO_NATURE_peptides = one_hot_encode_aa(new_TO_NATURE_peptides)
            # Convert the DataFrame to an H2OFrame and make the columns categorical.
            df_test = h2o.H2OFrame(pd.concat([new_TO_NATURE_peptides], axis='columns'))
        else:
            df_test = h2o.H2OFrame(pd.concat([new_TO_NATURE_peptides], axis='columns'))
            for column in df_test.columns:
                if column != training_column_name:
                    df_test[column] = df_test[column].asfactor()

        # Make predictions on the test data.
        predicted = best_model.predict(df_test).as_data_frame()
        new_TO_NATURE_peptides['predictions'] = predicted['predict'].to_list()

        # Concatenate the new predictions with the existing DataFrame.
        if len(data) == 0:
            data = new_TO_NATURE_peptides.copy()
        else:
            data = pd.concat([data, new_TO_NATURE_peptides], axis=0)
            data = data.sort_values('predictions', ascending=False)
            data = data[0:length_of_return_df]
    
    return data


def one_hot_encode_aa(df):
    """One-hot encode amino acid sequences in a pandas dataframe"""

    # Define amino acid alphabet
    aa_alphabet = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
                   'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-']

    # Determine maximum sequence length in input dataframe
    max_seq_len = df.applymap(len).max().max()

    # Initialize one-hot encoding dataframe
    encoding_df = pd.DataFrame()

    # Loop over columns in input dataframe and encode each amino acid sequence
    for col in df.columns:
        # Extract amino acid sequences from current column
        seqs = df[col].values

        # Initialize 2D array to hold one-hot encoding for current column
        encoding_matrix = np.zeros((len(seqs), max_seq_len * len(aa_alphabet)))

        # Loop over amino acid sequences and one-hot encode each one
        for i, seq in enumerate(seqs):
            # Get one-hot encoding vector for current amino acid sequence
            encoding_vec = np.zeros((max_seq_len, len(aa_alphabet)))
            for j, aa in enumerate(seq):
                encoding_vec[j, aa_alphabet.index(aa)] = 1

            # Flatten encoding vector and add it to encoding matrix for current column
            encoding_matrix[i, :] = encoding_vec.flatten()

        # Convert encoding matrix to dataframe and add it to one-hot encoding dataframe
        col_encoding_df = pd.DataFrame(encoding_matrix)
        col_encoding_df.columns = [f'{col}_{j}' for j in range(max_seq_len * len(aa_alphabet))]
        encoding_df = pd.concat([encoding_df, col_encoding_df], axis=1)

    return encoding_df


def one_hot_decode_aa(df, max_length = 22):
    # Define amino acid alphabet
    aa_alphabet = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M',
                    'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-']
    sps = []
    peptide = ''
    cols = list(df.columns)

    for index, row in df.iterrows():
        for col in cols: 
            if int(row[col]) ==1: 
                aa_index = int(col.split('_')[1])
                aa_acid = aa_alphabet[aa_index]
                peptide += aa_acid
                if len(peptide) == max_length: 
                    sps.append(peptide)
                    peptide = ''

    df['decoded_sequence'] = sps

    return df

# Gerating artificial Signal Peptides (SPs)

## 0 Introduction 
To avoid the combinatorial explosion that arises from the large number of possible amino acid sequences, an algorithm must be developed to narrow down the search space and identify the sequences that are most likely to function as signal peptides. This can be accomplished through a variety of computational methods, such as bioinformatics, machine learning, and statistical analysis.

One common approach is to use bioinformatics methods to analyze large sets of data on known signal peptides and identify patterns or features that are associated with signal peptide function. These features can then be used to predict the function of novel sequences.

Machine learning algorithms can also be used to predict signal peptides. These algorithms can be trained on large sets of data on known signal peptides, and can then be used to predict the function of novel sequences. Common machine learning algorithms used for this purpose include decision trees, random forests, and neural networks.

Another approach is to use statistical analysis to identify the regions of the peptide sequences that are most likely to function as signal peptides. This can be done by analyzing the frequency and distribution of different amino acids in known signal peptides and identifying those that are over-represented or under-represented in these sequences.

In summary, by developing an algorithm, we can narrow down the search space and identify the sequences that are most likely to function as signal peptides, thus avoiding combinatorial explosion. The algorithm we are showcasing here is based on a combination of bioinformatics, machine learning and statistical analysis.

### Agenda
- Use AutoML predictions and synthetic signal peptide generation algorotihm to get a novel list of potential signal peptides

## 1 Amino acid probability matrix

Lets import our df_pwn that was made in a previous notebook:

In [4]:
df_pwn = pd.read_csv('../data/02_all_signal_peptides/df_pwn_for_signal_peptides_found_in_supernatant.csv', index_col = False)
df_pwn

,A,C,D,E,F,G,H,I,K,L,...,N,P,Q,R,S,T,V,W,Y,-
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.070312,0.000000,0.003906,0.003906,0.035156,0.000000,0.062500,0.035156,0.234375,0.078125,...,0.011719,0.031250,0.070312,0.214844,0.031250,0.007812,0.046875,0.015625,0.015625,0.000000
2,0.042969,0.000000,0.003906,0.007812,0.179688,0.058594,0.000000,0.050781,0.015625,0.210938,...,0.023438,0.027344,0.023438,0.066406,0.140625,0.050781,0.066406,0.007812,0.015625,0.000000
3,0.070312,0.000000,0.003906,0.000000,0.097656,0.035156,0.015625,0.046875,0.062500,0.167969,...,0.031250,0.042969,0.058594,0.042969,0.179688,0.085938,0.015625,0.011719,0.019531,0.000000
4,0.109375,0.007812,0.007812,0.007812,0.035156,0.027344,0.015625,0.066406,0.066406,0.136719,...,0.046875,0.035156,0.046875,0.015625,0.167969,0.078125,0.046875,0.019531,0.046875,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.003906,0.000000,0.000000,0.000000,0.000000,0.992188
66,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094
67,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094
68,0.003906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.996094


In [5]:
amino_acids = list(df_pwn.columns.values)
amino_acids

['A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'Y',
 '-']

In [6]:
list_of_probabilities = []
for i in range(len(df_pwn)): 
    list_of_probabilities.append(df_pwn.loc[i, :].values.tolist())
list_of_probabilities[1]

[0.0703125,
 0.0,
 0.00390625,
 0.00390625,
 0.03515625,
 0.0,
 0.0625,
 0.03515625,
 0.234375,
 0.078125,
 0.03125,
 0.01171875,
 0.03125,
 0.0703125,
 0.21484375,
 0.03125,
 0.0078125,
 0.046875,
 0.015625,
 0.015625,
 0.0]

In [7]:
len(list_of_probabilities)

70

In [8]:
list_of_probabilities_random = list_of_probabilities.copy()

In [9]:
# GEnerating random signal peptides with equal distribution
for i in range(0,len(list_of_probabilities_random)): 
    for j in range(len(list_of_probabilities_random[i])):
        list_of_probabilities_random[i][j] = (100/21)/100

In [10]:
list_of_probabilities_random

[[0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616],
 [0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.047619047619047616,
  0.04761904761

In [11]:
generate_artificial_peptides

<function __main__.generate_artificial_peptides(list_of_probabilities: numpy.ndarray, amino_acids: numpy.ndarray, n_peptides: int, max_len=50) -> pandas.core.frame.DataFrame>

In [12]:
# random_peptides = generate_artificial_peptides(list_of_probabilities_random,amino_acids, 
#                                                 n_peptides=50000, 
#                                                 max_len=22)


# random_peptides_15_22 = random_peptides[random_peptides["length"] >= 15]

# random_peptides_15_22.to_csv('../data/05_best_signal_peptides/random_peptides_for_ML_training/random_peptides_for_ML_training_15_22.csv')
# random_peptides_15_22


## 2 Load best model from AutoML

In [13]:
h2o.init(ip="localhost", min_mem_size_GB=8)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_361"; Java(TM) SE Runtime Environment (build 1.8.0_361-b09); Java HotSpot(TM) 64-Bit Server VM (build 25.361-b09, mixed mode)
  Starting server from /Users/lucaslevassor/opt/anaconda3/envs/constrain/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/tmp0a150b2x
  JVM stdout: /var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/tmp0a150b2x/h2o_lucaslevassor_started_from_python.out
  JVM stderr: /var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/tmp0a150b2x/h2o_lucaslevassor_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,19 secs
H2O_cluster_timezone:,Europe/Copenhagen
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.1
H2O_cluster_version_age:,2 months and 22 days
H2O_cluster_name:,H2O_from_python_lucaslevassor_djoxyd
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.566 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [14]:
best_model = h2o.load_model("../data/04_ML_models/DeepLearning_grid_1_AutoML_1_20230426_90434_model_9")
best_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_1_20230426_90434_model_9


Status of Neuron Layers: predicting abundance, regression, gaussian distribution, Quadratic loss, 8,221 weights/biases, 143.7 KB, 1,176,000 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms            mean_bias            bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  --------------------  -------------------  -----------------------
    1        409      Input             0.0
    2        20       RectifierDropout  10.0       0.0   0.0   0.06183238344844711    0.09828370809555054    0.0         0.0686823157126141    0.3475421667098999    -0.6929061269026185  0.42695140838623047
    3        1        Linear                       0.0   0.0   9.380200580721976e-06  2.689490793272853e-05  0.0         0.005532642500475049  0.004399014636874199  -0.0648036890675343  1.0971281125650402e-154

ModelMetricsRegression: deeplearning
** Reported on train data. **

MSE: 0.0013813254682343238
RMSE: 0.03716618716298894
MAE: 0.004519562342447943
RMSLE: 0.02605423705509127
Mean Residual Deviance: 0.0013813254682343238

ModelMetricsRegression: deeplearning
** Reported on cross-validation data. **

MSE: 0.0015136262141688512
RMSE: 0.038905349428694905
MAE: 0.0037836424633789054
RMSLE: 0.027893721713893975
Mean Residual Deviance: 0.0015136262141688512

Cross-Validation Metrics Summary: 
                        mean        sd          cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid    cv_6_valid    cv_7_valid    cv_8_valid    cv_9_valid    cv_10_valid
----------------------  ----------  ----------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------
mae                     0.00376862  0.00537953  0.0184182     0.00592253    0.00270099    0.00121507    0.00162552    0.00344455    0.00101293    0.00136115    0.000845748   0.0011395
mean_residual_deviance  0.00150353  0.00409941  0.0130892     0.00157343    0.000200184   3.1951e-06    7.38857e-05   5.16834e-05   9.51947e-06   1.16178e-05   8.29618e-06   1.4242e-05
mse                     0.00150353  0.00409941  0.0130892     0.00157343    0.000200184   3.1951e-06    7.38857e-05   5.16834e-05   9.51947e-06   1.16178e-05   8.29618e-06   1.4242e-05
r2                      0.0103269   0.100405    0.0172692     -0.0225876    -0.0213516    0.283141      -0.0287704    0.0367059     -0.0717703    -0.0267836    -0.0447277    -0.0178563
residual_deviance       0.00150353  0.00409941  0.0130892     0.00157343    0.000200184   3.1951e-06    7.38857e-05   5.16834e-05   9.51947e-06   1.16178e-05   8.29618e-06   1.4242e-05
rmse                    0.0198944   0.0350831   0.114408      0.0396665     0.0141486     0.00178748    0.00859568    0.00718912    0.00308536    0.00340849    0.00288031    0.00377386
rmsle                   0.0156568   0.0242164   0.078839      0.034205      0.0133928     0.00178128    0.00831871    0.0070099     0.00306148    0.00337637    0.00285759    0.00372643

Scoring History: 
    timestamp            duration           training_speed    epochs    iterations    samples      training_rmse    training_deviance    training_mae    training_r2
--  -------------------  -----------------  ----------------  --------  ------------  -----------  ---------------  -------------------  --------------  -------------
    2023-04-26 09:31:39  0.000 sec                            0         0             0            nan              nan                  nan             nan
    2023-04-26 09:31:39  25 min 20.495 sec  55263 obs/sec     10        1             7350         0.042531         0.00180889           0.0150107       -0.1

## 3 Signal peptide predictor algorithm

The signal peptide predictor algorithm works by: 

    1. Generating artificial signal peptides - based on the PWM.
    2. It splits the artificial signal peptides into a dataframe adding '_' to the positions that are empty. 
    3. Initiates the H2O dataframe and makes all values except the feature column categorical
    4. Predicts the columns - based on a previously made ML model
    5. Makes a new dataframe with the best predictions, sorts it and slices out top100 
    6. The loop will continue until the specified number is reached. 
 

Note: Minimum number of iterations is 2.

In [15]:
signal_peptide_predictor

<function __main__.signal_peptide_predictor(list_of_probabilities: list, amino_acids: str, n_peptides: int, number_of_iterations: int, best_model: <module 'h2o.estimators' from '/Users/lucaslevassor/opt/anaconda3/envs/constrain/lib/python3.8/site-packages/h2o/estimators/__init__.py'>, training_column_name: str = 'MM_N_peptide_abundance', max_len_of_signal_peptides: int = 30, length_of_return_df=500, one_hot_encode: bool = True) -> pandas.core.frame.DataFrame>

In [16]:
%%capture
run = True

if run: 
    best_predicted_SPs = signal_peptide_predictor(list_of_probabilities, 
                                                        amino_acids, n_peptides =  5000,  
                                                        number_of_iterations = 10000, 
                                                        best_model = best_model, 
                                                        max_len_of_signal_peptides=22,
                                                        one_hot_encode = True,
                                                        length_of_return_df = 5000,
                                                        training_column_name = 'MM_N_peptide_abundance')

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Total input file size of 5.9 MB is much larger than total cluster memory of Zero  , please use either a larger cluster or smaller data.
  Request: POST /3/Parse
    data: {'destination_frame': 'Key_Frame__upload_b380c5285f8da7aa2997bdba57cd4972.hex', 'parse_type': 'CSV', 'separator': '44', 'check_header': '1', 'number_columns': '462', 'chunk_size': '4194304', 'delete_on_done': 'True', 'blocking': 'False', 'column_types': '["Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeri

In [ ]:
decoded_df = one_hot_decode_aa(best_predicted_SPs)

In [ ]:
decoded_df

,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,...,21_13,21_14,21_15,21_16,21_17,21_18,21_19,21_20,predictions,decoded_sequence
6909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108413,MMSANVWTLAYLVVSNAAALRE
8894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.098545,MVNKVVGSTACAAKSVTTPYVE
6114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094471,MKFLALWLASYGAQATPTSLSE
6545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094046,MHLAPLPSTYLFTQSLATPTAE
5433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093624,MKTASWLATLVLATRLMTLLRE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063457,MQLKELTVAIYLSIALPAFDAE
4484,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063449,MRGHLSSTGTCYLQFPLGATLE
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063448,MRLFGGLAALLGLASSISLQDE
6583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.063445,MRVTIRAFAATSLLATASLTAE


In [ ]:
best_predicted_SPs.to_csv('../data/05_best_signal_peptides/model_22_one_hot_encoded/encoded_i10000_npeptides5000.csv', index=False)
decoded_df.to_csv('../data/05_best_signal_peptides/model_22_one_hot_encoded/decoded_i10000_npeptides5000.csv', index=False)



In [ ]:
#best_predicted_SPs = pd.read_csv('../data/05_best_signal_peptides/model_22_/100_i/best_signal_peptides_i_100.csv', index_col=False)

In [ ]:
best_predicted_SPs.head(50)


,0_0,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,...,21_13,21_14,21_15,21_16,21_17,21_18,21_19,21_20,predictions,decoded_sequence
6909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.108413,MMSANVWTLAYLVVSNAAALRE
8894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.098545,MVNKVVGSTACAAKSVTTPYVE
6114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094471,MKFLALWLASYGAQATPTSLSE
6545,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.094046,MHLAPLPSTYLFTQSLATPTAE
5433,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093624,MKTASWLATLVLATRLMTLLRE
3254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093499,MKGSCIESTACIAISAAIPVTE
2526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.093122,MRLAGLLDSLLLVFAAATPLAE
2742,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092970,MKSRSSLATKSTLGSALTPLVE
8072,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092966,MFTNLLLTSLPGAQSAAAYLGE
9575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092918,MTSKAFFFAALSVFSAAVPLRE


In [ ]:
from Bio.SeqRecord import SeqRecord
predicted_signal_peptide_sequences = best_predicted_SPs['decoded_sequence'].to_list()
predicted_signal_peptide_sequences
#predicted_signal_peptide_sequences = [SeqRecord(peptide) for peptide in predicted_signal_peptide_sequences]

['MMSANVWTLAYLVVSNAAALRE',
 'MVNKVVGSTACAAKSVTTPYVE',
 'MKFLALWLASYGAQATPTSLSE',
 'MHLAPLPSTYLFTQSLATPTAE',
 'MKTASWLATLVLATRLMTLLRE',
 'MKGSCIESTACIAISAAIPVTE',
 'MRLAGLLDSLLLVFAAATPLAE',
 'MKSRSSLATKSTLGSALTPLVE',
 'MFTNLLLTSLPGAQSAAAYLGE',
 'MTSKAFFFAALSVFSAAVPLRE',
 'MKIAAMSLFMYGLLVWAQLTAE',
 'MKVFAAVLLLLLPTSALVALRE',
 'MFILAATMFVDTYLSASTLLAE',
 'MKTNSISSTLALAVSLESSLAE',
 'MKTKAALAVAYICQASAAAPAE',
 'MMFSYLLAALTAVTSYAVPLPE',
 'MLFSLSFSLAFSLVVAAAPLTE',
 'MFSSILSSLRSLLQAAEPALAE',
 'MKKLNLAAQAALLKSALAPYAE',
 'MRGNYAFSTLFALGSLAAHADE',
 'MPSTTSFTTLSLLQVPSAPAAE',
 'MKGFSAVDTASMKSALAAALVE',
 'MAPRAVSALQCAGQWVASNPDE',
 'MMIANLIALGPLLVPSAAYARE',
 'MLMNYTSTTATGCTPTPTAAAE',
 'MVRSAIALQGYLAQRAATAYLE',
 'MRRAHSTSLATLLQSSEPAAAE',
 'MATLASWAMSLAATVFVHNCVE',
 'MRGTYLTTLAAAALSGAAPQVE',
 'MKLFAALATLTIGVVPSTSLVE',
 'MRLFAKLTSLYLPQCVVVALTE',
 'MMLNAIISTLLLLLASWFAAAE',
 'MVRSSAALALLGATLLVAPLAE',
 'MKKNHTESSGLASASSFTFWHE',
 'MFGSNGLTSLTALVFLFTCARE',
 'MRNAPWLFLAATASPVAAPPAE',
 'MKIQAFFGMASFLVASETSLTE',
 

In [ ]:
from Bio.Seq import Seq 

In [ ]:
# add gfp
GFP = 'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'
predicted_signal_peptide_sequences = [Seq(peptide+GFP)  for peptide in predicted_signal_peptide_sequences]


In [ ]:
# Make to SeqRecord
predicted_signal_peptide_sequences = [SeqRecord(peptide)  for peptide in predicted_signal_peptide_sequences]
#predicted_signal_peptide_sequences = [predicted_signal_peptide_sequences[i].id = i  for peptide in range(len(predicted_signal_peptide_sequences))]

In [ ]:
predicted_signal_peptide_sequences

[SeqRecord(seq=Seq('MMSANVWTLAYLVVSNAAALREMSKGEELFTGVVPILVELDGDVNGHKFSVSGE...LYK'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('MVNKVVGSTACAAKSVTTPYVEMSKGEELFTGVVPILVELDGDVNGHKFSVSGE...LYK'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('MKFLALWLASYGAQATPTSLSEMSKGEELFTGVVPILVELDGDVNGHKFSVSGE...LYK'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('MHLAPLPSTYLFTQSLATPTAEMSKGEELFTGVVPILVELDGDVNGHKFSVSGE...LYK'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('MKTASWLATLVLATRLMTLLREMSKGEELFTGVVPILVELDGDVNGHKFSVSGE...LYK'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('MKGSCIESTACIAISAAIPVTEMSKGEELFTGVVPILVELDGDVNGHKFSVSGE...LYK'), id='<unknown id>', name='<unknown na

In [ ]:
for i in range(len(predicted_signal_peptide_sequences)):
    predicted_signal_peptide_sequences[i].id = str(i+1)
    predicted_signal_peptide_sequences[i].name = f'sp{i+1:00}'

In [ ]:
predicted_signal_peptide_sequences[i]

SeqRecord(seq=Seq('MRRLALCLLLMLSVGVRADQHEMSKGEELFTGVVPILVELDGDVNGHKFSVSGE...LYK'), id='5000', name='sp5000', description='<unknown description>', dbxrefs=[])

In [ ]:
#for lenght in predicted_signal_peptide_sequences:
#    print(len(lenght))

In [ ]:
from Bio import SeqIO

with open(f"../data/05_best_signal_peptides/best_signal_peptides_as_fasta/signal_peptides1.fasta", "w") as output_handle:
    SeqIO.write(predicted_signal_peptide_sequences, output_handle, "fasta")

In [ ]:
predicted_signal_peptide_sequences

[SeqRecord(seq='MMSANVWTLAYLVVSNAAALRE', id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq='MVNKVVGSTACAAKSVTTPYVE', id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq='MKFLALWLASYGAQATPTSLSE', id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq='MHLAPLPSTYLFTQSLATPTAE', id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq='MKTASWLATLVLATRLMTLLRE', id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq='MKGSCIESTACIAISAAIPVTE', id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq='MRLAGLLDSLLLVFAAATPLAE', id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq='MKSRSSLATKSTLGSALTPLVE', id='<unknown id>', name='<un

In [ ]:
# new_TO_NATURE_peptides = generate_artificial_peptides(list_of_probabilities, 
#                                                         amino_acids, 
#                                                         n_peptides=1000, 
#                                                         max_len=22)

# new_TO_NATURE_peptides_split = split_peptides_sequences(new_TO_NATURE_peptides)
# one_hot_peptides = one_hot_encode_aa(new_TO_NATURE_peptides_split)
# df_w_decoded_seqs = one_hot_decode_aa(one_hot_peptides)
# df_w_decoded_seqs

In [ ]:
# decode_sequences = list(df_w_decoded_seqs['decoded_sequence'])
# ref_seqs =  list(new_TO_NATURE_peptides['sequence'])



# count = 0
# for seq in decode_sequences: 
#     if seq in ref_seqs: 
#         count += 1

# if count == len(decode_sequences): 
#     print('successful')


### Get the best peptide sequence

In [ ]:
# list_of_aa = []
# for i in range(0,len(best_predicted_SPs)):
#     #signal_peptide = "".join(best_predicted_SPs.iloc[i][:-1].tolist())
#     peptide = best_predicted_SPs.iloc[i][:-1].tolist()

#     #peptide_wo_dash = [aa for aa in peptide if aa != '-']
#     #if '-' in peptide: 
#     list_of_aa.append(peptide)
    
# len(list_of_aa)

In [ ]:
#list_of_aa.append(aa1)

In [ ]:
#cols = [str(i) for i in range(0,22)]

In [ ]:
#best_predicted_SPs['sequence'] = best_predicted_SPs[cols].agg(''.join, axis=1)
#best_predicted_SPs

In [ ]:
#best_predicted_SPs = best_predicted_SPs.replace(to_replace = '-', value ='')
#best_predicted_SPs['sequence'] =  [string.replace("-", "") for string in list(best_predicted_SPs['sequence'])]

KeyError: 'sequence'

In [ ]:
lenght_of_signal_peptides = []
for k,v in best_predicted_SPs.iterrows(): 
    #print(v['sequence'])
    lenght_of_signal_peptides.append(len(v['sequence']))
best_predicted_SPs['length'] = lenght_of_signal_peptides

In [ ]:
best_predicted_SPs

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,predictions,sequence,length
0,M,K,F,S,A,I,S,V,L,A,...,L,S,A,G,Q,Q,A,0.032961,MKFSAISVLACIGTSLSAGQQA,22
1,M,M,F,R,Y,I,F,L,G,V,...,A,A,T,A,A,V,A,0.032511,MMFRYIFLGVLALTAAATAAVA,22
2,M,L,F,L,A,A,L,T,L,A,...,G,A,A,A,L,I,P,0.032439,MLFLAALTLALCGYPGAAALIP,22
3,M,K,F,G,A,L,A,S,S,G,...,L,A,A,S,L,V,T,0.030866,MKFGALASSGAGALALAASLVT,22
4,M,R,V,S,A,A,L,T,I,G,...,Y,S,A,F,L,T,E,0.030141,MRVSAALTIGAAMLNYSAFLTE,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,M,H,F,L,A,T,V,S,L,L,...,A,V,A,A,L,H,A,0.019090,MHFLATVSLLCLASAAVAALHA,22
96,M,A,I,Q,A,P,F,S,W,G,...,R,A,A,D,L,A,A,0.019026,MAIQAPFSWGGPLAVRAADLAA,22
97,M,R,F,S,K,A,V,F,S,A,...,A,A,A,A,P,I,V,0.018995,MRFSKAVFSAGAGLSAAAAPIV,22
98,M,K,N,P,S,F,L,P,T,A,...,L,A,F,A,Q,N,A,0.018968,MKNPSFLPTACMLIALAFAQNA,22


In [ ]:
SPs_22 = best_predicted_SPs[(best_predicted_SPs[['length']]<23).all(axis=1)]
SPs_22

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,predictions,sequence,length
0,M,L,R,F,S,A,S,P,L,G,...,,,,,,,,0.194617,MLRFSASPLGYKGQAAAADQ,20
1,M,M,R,F,S,W,I,A,F,L,...,,,,,,,,0.185066,MMRFSWIAFLSLLVATAP,18
2,M,V,A,A,S,A,S,F,L,I,...,,,,,,,,0.183458,MVAASASFLILSLVAYLAAQV,21
3,M,V,R,H,S,A,I,F,W,A,...,,,,,,,,0.183349,MVRHSAIFWAAALQVTLP,18
4,M,L,R,F,S,L,F,F,G,A,...,,,,,,,,0.178592,MLRFSLFFGACILQFHAP,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,M,L,R,F,A,L,L,L,F,L,...,,,,,,,,0.142601,MLRFALLLFLCSLLSAAA,18
95,M,V,S,V,I,L,I,G,F,L,...,,,,,,,,0.142263,MVSVILIGFLLGLYSTGATV,20
96,M,M,R,F,S,S,L,L,F,L,...,,,,,,,,0.142099,MMRFSSLLFLNMLLIAPAALI,21
97,M,L,R,F,M,V,S,A,M,A,...,,,,,,,,0.141988,MLRFMVSAMASALYALLASL,20


In [ ]:
#SPs_22.to_csv('../data/05_best_signal_peptides/10_000/best_signal_peptides_i_10_000_w_sps_22.csv', index=False)


In [ ]:
h2o.shutdown()

/var/folders/2f/lw3sfzbs7l7f_q1knzbtbwrr0000gp/T/ipykernel_832/2900054706.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


H2O session _sid_a966 closed.
